In [2]:
#Block 1
# Importing dependencies to clean data
import requests
import numpy as np
import pandas as pd
from config import fmp_api_key

# variable for ticker symbol
ticker = 'AAPL'

# API Key (DO NOT PUSH UNTIL HIDDEN)


# API request for income statement variable
IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={fmp_api_key}').json()

# calculates revvenue growth from previous year (Would like to expand this to get 
# average across last 5 years to make a ten year model)
revenue_growth = []   
revenue_growth = (IS[0]['revenue'] - IS[1]['revenue'])/ IS[1]['revenue']

print(revenue_growth)

# Creates Percentage of Sales Income Statement
# varable for next income in the income statement
net_income = IS[0]['netIncome']

# Block 2

# Cleaning income statement call into a dataframe and 
# calculating the income statement as a percentage of revenue
income_statement = pd.DataFrame.from_dict(IS[0], orient = 'index')
income_statement = income_statement[6:-2] 
income_statement.columns = ['current_year']
income_statement['as_%_of_revenue'] = income_statement / income_statement.iloc[0]

# Forecasting income statement growth for next 5 years
income_statement['next_year'] = (income_statement['current_year']['revenue'] * (1+revenue_growth)) *income_statement['as_%_of_revenue']
income_statement['next_2_years'] = (income_statement['next_year']['revenue'] * (1+revenue_growth)) *income_statement['as_%_of_revenue']
income_statement['next_3_years'] = (income_statement['next_2_years']['revenue'] * (1+revenue_growth)) *income_statement['as_%_of_revenue']
income_statement['next_4_years'] = (income_statement['next_3_years']['revenue'] * (1+revenue_growth)) *income_statement['as_%_of_revenue']
income_statement['next_5_years'] = (income_statement['next_4_years']['revenue'] * (1+revenue_growth)) *income_statement['as_%_of_revenue']

# Block 3

# API Request for balance sheet statement
BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={fmp_api_key}').json()

# Cleans Balance Sheet
balance_sheet = pd.DataFrame.from_dict(BS[0],orient='index')
balance_sheet = balance_sheet[6:-2]
balance_sheet.columns = ['current_year']
balance_sheet['as_%_of_revenue'] = balance_sheet / income_statement['current_year'].iloc[0]
   

# Forecasts as percentage of revenue
balance_sheet['next_year'] = income_statement['next_year'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_2_years'] = income_statement['next_2_years'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_3_years'] = income_statement['next_3_years'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_4_years'] = income_statement['next_4_years'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_5_years'] = income_statement['next_5_years'] ['revenue'] * balance_sheet['as_%_of_revenue']

# Forecasts Free Cash Flows, Net Income, Depreciation, Receivables, Inventory, Payables, Cash from Operations, & CAPEX 
# for Next Year
CF_forecast = {}
CF_forecast['next_year'] = {}

CF_forecast['next_year']['netIncome'] = income_statement['next_year']['netIncome']
CF_forecast['next_year']['inc_depreciation'] = income_statement['next_year']['depreciationAndAmortization']
CF_forecast['next_year']['inc_receivables'] = balance_sheet['next_year']['netReceivables'] - balance_sheet['current_year']['netReceivables']
CF_forecast['next_year']['inc_inventory'] = balance_sheet['next_year']['inventory'] - balance_sheet['current_year']['inventory']
CF_forecast['next_year']['inc_payables'] = balance_sheet['next_year']['accountPayables'] - balance_sheet['current_year']['accountPayables']
CF_forecast['next_year']['CF_operations'] = CF_forecast['next_year']['netIncome'] + CF_forecast['next_year']['inc_depreciation'] + (CF_forecast['next_year']['inc_receivables'] * -1) + (CF_forecast['next_year']['inc_inventory'] *-1) + CF_forecast['next_year']['inc_payables']
CF_forecast['next_year']['CAPEX'] = balance_sheet['next_year']['propertyPlantEquipmentNet'] - balance_sheet['current_year']['propertyPlantEquipmentNet'] + income_statement['next_year']['depreciationAndAmortization']
CF_forecast['next_year']['FCF'] = CF_forecast['next_year']['CAPEX'] + CF_forecast['next_year']['CF_operations']

# # Forecasts Free Cash Flows, Net Income, Depreciation, Receivables, Inventory, Payables, Cash from Operations, & CAPEX 
# # for Next Two Years
CF_forecast['next_2_years'] = {}
CF_forecast['next_2_years']['netIncome'] = income_statement['next_2_years']['netIncome']

CF_forecast['next_2_years']['inc_depreciation'] = income_statement['next_2_years']['depreciationAndAmortization']
CF_forecast['next_2_years']['inc_receivables'] = balance_sheet['next_2_years']['netReceivables'] - balance_sheet['next_year']['netReceivables']
CF_forecast['next_2_years']['inc_inventory'] = balance_sheet['next_2_years']['inventory'] - balance_sheet['next_year']['inventory']
CF_forecast['next_2_years']['inc_payables'] = balance_sheet['next_2_years']['accountPayables'] - balance_sheet['next_year']['accountPayables']
CF_forecast['next_2_years']['CF_operations'] = CF_forecast['next_2_years']['netIncome'] + CF_forecast['next_2_years']['inc_depreciation'] + (CF_forecast['next_2_years']['inc_receivables'] * -1) + (CF_forecast['next_2_years']['inc_inventory'] *-1) + CF_forecast['next_2_years']['inc_payables']
CF_forecast['next_2_years']['CAPEX'] = balance_sheet['next_2_years']['propertyPlantEquipmentNet'] - balance_sheet['next_year']['propertyPlantEquipmentNet'] + income_statement['next_2_years']['depreciationAndAmortization']
CF_forecast['next_2_years']['FCF'] = CF_forecast['next_2_years']['CAPEX'] + CF_forecast['next_2_years']['CF_operations']

# Forecasts Free Cash Flows, Net Income, Depreciation, Receivables, Inventory, Payables, Cash from Operations, & CAPEX 
# for Next Three Years
CF_forecast['next_3_years'] = {}
CF_forecast['next_3_years']['netIncome'] = income_statement['next_3_years']['netIncome']

CF_forecast['next_3_years']['inc_depreciation'] = income_statement['next_3_years']['depreciationAndAmortization'] 
CF_forecast['next_3_years']['inc_receivables'] = balance_sheet['next_3_years']['netReceivables'] - balance_sheet['next_2_years']['netReceivables']
CF_forecast['next_3_years']['inc_inventory'] = balance_sheet['next_3_years']['inventory'] - balance_sheet['next_2_years']['inventory']
CF_forecast['next_3_years']['inc_payables'] = balance_sheet['next_3_years']['accountPayables'] - balance_sheet['next_2_years']['accountPayables']
CF_forecast['next_3_years']['CF_operations'] = CF_forecast['next_3_years']['netIncome'] + CF_forecast['next_3_years']['inc_depreciation'] + (CF_forecast['next_3_years']['inc_receivables'] * -1) + (CF_forecast['next_3_years']['inc_inventory'] *-1) + CF_forecast['next_3_years']['inc_payables']
CF_forecast['next_3_years']['CAPEX'] = balance_sheet['next_3_years']['propertyPlantEquipmentNet'] - balance_sheet['next_2_years']['propertyPlantEquipmentNet'] + income_statement['next_3_years']['depreciationAndAmortization']
CF_forecast['next_3_years']['FCF'] = CF_forecast['next_3_years']['CAPEX'] + CF_forecast['next_3_years']['CF_operations']

# Forecasts Free Cash Flows, Net Income, Depreciation, Receivables, Inventory, Payables, Cash from Operations, & CAPEX 
# for Next Four Years

CF_forecast['next_4_years'] = {}
CF_forecast['next_4_years']['netIncome'] = income_statement['next_4_years']['netIncome']

CF_forecast['next_4_years']['inc_depreciation'] = income_statement['next_4_years']['depreciationAndAmortization']
CF_forecast['next_4_years']['inc_receivables'] = balance_sheet['next_4_years']['netReceivables'] - balance_sheet['next_3_years']['netReceivables']
CF_forecast['next_4_years']['inc_inventory'] = balance_sheet['next_4_years']['inventory'] - balance_sheet['next_3_years']['inventory']
CF_forecast['next_4_years']['inc_payables'] = balance_sheet['next_4_years']['accountPayables'] - balance_sheet['next_3_years']['accountPayables']
CF_forecast['next_4_years']['CF_operations'] = CF_forecast['next_4_years']['netIncome'] + CF_forecast['next_4_years']['inc_depreciation'] + (CF_forecast['next_4_years']['inc_receivables'] * -1) + (CF_forecast['next_4_years']['inc_inventory'] *-1) + CF_forecast['next_4_years']['inc_payables']
CF_forecast['next_4_years']['CAPEX'] = balance_sheet['next_4_years']['propertyPlantEquipmentNet'] - balance_sheet['next_3_years']['propertyPlantEquipmentNet'] + income_statement['next_4_years']['depreciationAndAmortization']
CF_forecast['next_4_years']['FCF'] = CF_forecast['next_4_years']['CAPEX'] + CF_forecast['next_4_years']['CF_operations']

# Forecasts Free Cash Flows, Net Income, Depreciation, Receivables, Inventory, Payables, Cash from Operations, & CAPEX 
# for Next Five Years

CF_forecast['next_5_years'] = {}
CF_forecast['next_5_years']['netIncome'] = income_statement['next_5_years']['netIncome']

CF_forecast['next_5_years']['inc_depreciation'] = income_statement['next_5_years']['depreciationAndAmortization'] 
CF_forecast['next_5_years']['inc_receivables'] = balance_sheet['next_5_years']['netReceivables'] - balance_sheet['next_4_years']['netReceivables']
CF_forecast['next_5_years']['inc_inventory'] = balance_sheet['next_5_years']['inventory'] - balance_sheet['next_4_years']['inventory']
CF_forecast['next_5_years']['inc_payables'] = balance_sheet['next_5_years']['accountPayables'] - balance_sheet['next_4_years']['accountPayables']
CF_forecast['next_5_years']['CF_operations'] = CF_forecast['next_5_years']['netIncome'] + CF_forecast['next_5_years']['inc_depreciation'] + (CF_forecast['next_5_years']['inc_receivables'] * -1) + (CF_forecast['next_5_years']['inc_inventory'] *-1) + CF_forecast['next_5_years']['inc_payables']
CF_forecast['next_5_years']['CAPEX'] = balance_sheet['next_5_years']['propertyPlantEquipmentNet'] - balance_sheet['next_4_years']['propertyPlantEquipmentNet'] + income_statement['next_5_years']['depreciationAndAmortization']
CF_forecast['next_5_years']['FCF'] = CF_forecast['next_5_years']['CAPEX'] + CF_forecast['next_5_years']['CF_operations']

# Creating Cash Flow Forecast Data Frame
CF_forec = pd.DataFrame.from_dict(CF_forecast, orient='columns')
pd.options.display.float_format='{:,.0f}'.format

print(CF_forec)

# WACC = Kd * (1 -Tc) * (D /D+E) + Ke * (E /D+E)
# WACC = Weighted Average Cost of Capital
# Kd = Cost of debt
# Tc = Firm tax rate
# Ke = Cost of equity
# D / D + E = Proportion of debt in firm capital structure
# REF: https://codingandfun.com/calculating-weighted-average-cost-of-capital-wacc-with-python/

import requests
import pandas_datareader.data as web
import datetime

# Function one (Interest Coverage Ratio and Risk Free Rate of Return)
# compares the EBIT & Interest Expense, then turns the 1 year treasury note into percentage.
# Store Risk free rate and interest coverage ratio into a variable to be passed along into Cost of Debt Function

def interest_coverage_and_RF(ticker):

# Requesting and Clearning EBIT
    IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={fmp_api_key}').json()
    EBIT = IS[0]['ebitda'] - IS[0]['depreciationAndAmortization']
    
# Analyzing interest_coverage_ratio
    interest_expense = IS[0]['interestExpense']
    interest_coverage_ratio = EBIT / interest_expense

# Establishing datetime for treasury bills
    start = datetime.datetime(2011,1,1)
    end = datetime.datetime.today().strftime('%Y-%m-%d')

# Reading treasury bill rates
    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)

# Isolating Treasury rate
    RF = float(Treasury.iloc[-1])

# Converting rate into percentage
    RF = RF/100
    
# returning list of Risk Free Rate, and Interest Coverage Ratio
#     print(RF)
    return [RF, interest_coverage_ratio]

    
# Storing Value of function.
# interest_coverage_and_RF(ticker)

# Function two (Cost of Debt (kd)) 
# Caculates the cost of debt using the company's credit spread. 
# This is in part because it is tedious to derive the cost of equity from interest paid on each individual 
# bond of a company, and data is not readily available. 
def cost_of_debt(ticker, RF, interest_coverage_ratio):
    if interest_coverage_ratio > 8.5:
    #Rating is AAA
        credit_spread = 0.0063
    if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
    #Rating is AA
        credit_spread = 0.0078
    if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
    #Rating is A+
        credit_spread = 0.0098
    if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
    #Rating is A
        credit_spread = 0.0108
    if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
    #Rating is A-
        credit_spread = 0.0122
    if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
    #Rating is BBB
        credit_spread = 0.0156
    if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
    #Rating is BB+
        credit_spread = 0.02
    if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
        credit_spread = 0.0240
    if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
        credit_spread = 0.0351
    if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
        credit_spread = 0.0421
    if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
        credit_spread = 0.0515
    if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
        credit_spread = 0.0820
    if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
        credit_spread = 0.0864
    if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
        credit_spread = 0.1134
    if interest_coverage_ratio <=  0.2:
    #Rating is D
        credit_spread = 0.1512

# Adding Risk Free Rate into Credit Spread
    cost_of_debt = RF + credit_spread
    return cost_of_debt

# Function two (Cost of Equity (ke))
def cost_of_equity(ticker):

# Establishing datetime for range of 1 year treasury notes
    start = datetime.datetime(2020,3,20)
    end = datetime.datetime.today().strftime('%Y-%m-%d')
    
# Reading treasury bill rates
    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)

# Isolating Risk Free Rate
    RF = float(Treasury.iloc[-1])
    
# Turning Risk Free Rate into percentage
    RF = RF/100

# Requesting and converting Beta into a float
    beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{ticker}?apikey={fmp_api_key}').json()
    beta = float(beta['profile']['beta'])
    
# establishing date range for S&P 500
    start = datetime.datetime(2020,3,20)
    end = datetime.datetime.today().strftime('%Y-%m-%d')

# Reading data from date range, and droping 0 values
    SP500 = web.DataReader(['sp500'], 'fred', start, end)
    SP500.dropna(inplace = True)
    
# Variable for S&P Yearly returns
    SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-252])-1
    
# Calculating and returning cost of equity
    cost_of_equity = RF + (beta *(SP500yearlyreturn - RF))
    return cost_of_equity

# variable for cost of equity function
# ke = cost_of_equity(ticker)

# Function Three (Tax Rate, Capital Structure and calculate WAAC)
# Generates Weighted Average Cost of Capital for the company to show the expected growth of the 
# company based on company's capital strucuture and weighted for inflation.

def wacc(ticker):
    
# Gathering Effective Tax Rate
    FR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={fmp_api_key}').json()
    ETR = FR[0]['effectiveTaxRate']

# AREA OF CONCERN: line 2 & 3
# Requests balance sheet and divides total debt 
    BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=annual&apikey={fmp_api_key}').json()

# Proportion of debt in firm capital strucuture    
    Debt_to = BS[0]['totalDebt']/ (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity'])

# Proportion of equity in firms capital strucuture
    equity_to = BS[0]['totalStockholdersEquity'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity'])
    
# Calculating WACC
    WACC = (kd*(1-ETR)*Debt_to) + (ke*equity_to)
    return WACC

# Storing WACC function as a variable
RF_and_IntCov = interest_coverage_and_RF(ticker)
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = cost_of_equity(ticker)
kd = cost_of_debt(ticker, RF, interest_coverage_ratio)
wacc_company = wacc(ticker)

# print statement
print('The Weighted Average Cost of Capital for ' + ticker + ' is ' + str((wacc_company*100))+'%')

#calculates company's net present value of free cash flow. 
import numpy_financial as npf

FCF_List = CF_forec.iloc[-1].values.tolist()
npv = npf.npv(wacc_company,FCF_List)

#Terminal value
LTGrowth = 0.025

Terminal_value = (CF_forecast['next_5_years']['FCF'] * (1+ LTGrowth)) /(wacc_company  - LTGrowth)

Terminal_value_Discounted = Terminal_value/(1+wacc_company)**4

target_equity_value = Terminal_value_Discounted + npv
debt = balance_sheet['current_year']['totalDebt']
target_value = target_equity_value - debt
numbre_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{ticker}?apikey={fmp_api_key}').json()
numbre_of_shares = numbre_of_shares[0]['numberOfShares']

target_price_per_share = target_value/numbre_of_shares
target_price_per_share

print(ticker + ' forecasted price per stock is ' + str(target_price_per_share) )
print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_growth) + ' Cost of Capital: ' + str(wacc_company) )
print('perpetuity growth: ' + str(LTGrowth))

0.055120803769784836
                      next_year   next_2_years   next_3_years    next_4_years  \
netIncome        60,575,540,465 63,914,512,944 67,437,532,271  71,154,743,254   
inc_depreciation 11,665,415,606 12,308,422,691 12,986,872,843  13,702,719,712   
inc_receivables   2,063,998,497  2,177,767,753  2,297,808,062   2,424,465,090   
inc_inventory       223,845,584    236,184,133    249,202,792     262,939,050   
inc_payables      2,331,389,516  2,459,897,580  2,595,489,112   2,738,554,558   
CF_operations    72,284,501,507 76,268,881,330 80,472,883,371  84,908,613,384   
CAPEX            13,691,987,078 14,446,700,411 15,243,014,149  16,083,221,341   
FCF              85,976,488,585 90,715,581,741 95,715,897,521 100,991,834,726   

                    next_5_years  
netIncome         75,076,849,894  
inc_depreciation  14,458,024,637  
inc_receivables    2,558,103,554  
inc_inventory        277,432,462  
inc_payables       2,889,505,887  
CF_operations     89,588,844,401  
CAPE